In [4]:
import codecs
import numpy as np
from pprint import pprint
from numpy.linalg import norm

In [14]:
#process centroids
# infile = "/Users/curtisxuan/Desktop/Work/KNN_Users/centroids.txt"
infile = "/Users/curtisxuan/Desktop/Work/experiment2/centroidsD.txt"
centroids = []
with codecs.open(infile, "r", "utf-8") as f:
    for line in f:
        dim = line.rstrip().split(" ")
        centroids.append(dim)
centroids = np.array (centroids,dtype='float32')
centroids = centroids[:,1:]
# print(centroids)

#parse features
featureVec = "/Users/curtisxuan/Desktop/Work/experiment2/embedding_d.txt"
count=0
features=[]
category=[]
with codecs.open(featureVec, 'r', 'utf8') as f:
    for line in f:
        if count==0:
            count+=1
            continue
        info = line.rstrip().split('\t')
        category.append(info[0])
        vector = [float(v) for v in info[3].split(' ')]
        features.append(vector)
rawFea = np.array (features,dtype='float32')
# normalize features
L2norm = norm(rawFea, axis=1, ord=2)

rawFeatures=[]
count=0
categ=[]
for i in L2norm:
    if i!=0:    
        rawFeatures.append(rawFea[count]/i)
        categ.append(category[count])
    count+=1
# print(features[0])
# print(rawFeatures[0])
# print(categ[0])

temp1 = categ
temp2 = rawFeatures
categ = []
rawFeatures=[]
for i in range(len(temp1)):
    if temp1[i].startswith('doc'):
        categ.append(temp1[i])
        rawFeatures.append(temp2[i])
# print(len(categories))        
# for i in categories:
#     rawCategories.append(i[1:])
# rawCategories = np.array (rawCategories,dtype='float32')
# normalize category
# L2norm = norm(rawCategories, axis=1, ord=2)
# rawCategories=rawCategories / L2norm.reshape(127,1)
# print(categories[0])
# print(rawCategories[0])


# #extract ONLY CATEGORIES from features 
# rl = ['ct#poi','ct#twitter','ct#smartnews','ct#googlenews','ct#flipboard']
# categories=[]
# rawCategories=[]
# for i in features:
#     if (i[0].startswith('tct#') or (i[0].startswith('ct#') and (not '_' in i[0]) and (not (i[0].startswith(rl[0]) or i[0].startswith(rl[1]) or i[0].startswith(rl[2]) or i[0].startswith(rl[3]) or i[0].startswith(rl[4]))))):
#         categories.append(i)
# # print(len(categories))        
# for i in categories:
#     rawCategories.append(i[1:])
# rawCategories = np.array (rawCategories,dtype='float32')
# normalize category
# L2norm = norm(rawCategories, axis=1, ord=2)
# rawCategories=rawCategories / L2norm.reshape(127,1)
# print(categories[0])
# print(rawCategories[0])

In [15]:
#find top 5 features for each centroid
import requests
from lxml.html import fromstring

ct=1
minList=[]
for centroid in centroids:
    tempRaw = rawFeatures
    tempFea = categ
    top5 = []
    for iteration in range(1,6):
        count=0
        minDist=100
        minIndex=0
        for feature in tempRaw:
            dist = np.linalg.norm(centroid-feature)
        #         print(dist)
            if minDist > dist:
                minDist = dist
                minIndex = count
            count+=1
        if tempFea[minIndex].startswith('doc'):
            code = tempFea[minIndex][6:]
            r = requests.get('http://www.newsbreakapp.com/news/'+code)
            tree = fromstring(r.content)
            title = tree.findtext('.//title')
            top5.append([iteration,code +': ' + title,minDist])
            tempRaw=np.delete(tempRaw,minIndex,0)
            tempFea.pop(minIndex)
            
        else:
            top5.append([iteration,tempFea[minIndex],minDist])
            tempRaw=np.delete(tempRaw,minIndex,0)
            tempFea.pop(minIndex)
    print ([ct,top5])
    minList.append([ct,top5])
    ct+=1
pprint(minList)



[1, [[1, "0MBHjSmC: Dr. Pimple Popper Squeezes 'Ground Beef' Out of Two Giant Back Cysts | News Break", 0.6547926], [2, '0LzolsHg: Watch Dr. Pimple Popper Get Sprayed on by Some Juicy Cysts in a New Instagram Video | News Break', 0.67727894], [3, '0M58CR0H: 13 Dr. Pimple Popper Videos That Are Deeply Satisfying | News Break', 0.8310742], [4, "0MCk7Byy: Dr. Pimple Popper Just Cleared This Man's Blackhead-Covered Forehead And I'm In Awe | News Break", 0.8344992], [5, '0M99jEf6: Watch Dr. Pimple Popper Treat an Inflamed Blackhead in This Wildly Popular YouTube Video | News Break', 0.85507876]]]
[2, [[1, "0MCi6jpn: Alabama: pregnant woman shot in stomach is charged in fetus's death | News Break", 0.78824586], [2, '0MCTJpkI: ‘Monster’ Sentenced To Life In Prison For 2011 Murders Of Sisters In Dorchester | News Break', 0.80281264], [3, '0MCNhOQ9: Who is Moise Kean and why he’d be perfect for Everton | News Break', 0.8125719], [4, '0MCKSqRb: Asylum officers union opposes sending migrants back

[16, [[1, '0MCgSPoY: Sources: Celts hiring Kara Lawson as assistant | News Break', 0.848693], [2, '0MCFXyBT: Newly defined cancer driver is fast, furious and loud | News Break', 0.85819197], [3, '0MCa2LzB: The 2020 Dodge Charger SRT Hellcat Widebody Flares Up | News Break', 0.8671046], [4, "0MCOz81X: 2020 Ford Super Duty Tremor Off-Road Package: Taking the Fight to Ram's Power Wagon | News Break", 0.87196136], [5, '0MCVUJXv: Heyman and Bischoff fill new WWE executive creative roles under Vince McMahon | News Break', 0.8774225]]]
[17, [[1, '0MBsisQy: Democrats face dilemma over Iran | News Break', 0.8012], [2, '0MBs2JDy: Power Rangers Necessary Evil Team Talks Big Reveals, Rangers, Villains, and More | News Break', 0.8285357], [3, '0MCAkrbu: Shop The Kate Spade New York x Refinery29 Pilot Season Short Films | News Break', 0.8301215], [4, '0MCmDCq6: A Delaware Grandmother Vanished 2 Weeks Ago, and Her Family Is Desperate for Answers | News Break', 0.83275276], [5, '0MAwjNcV: The Secret B

[31, [[1, '0MAuYGlP: Injury Update For USWNT Star Forward Alex Morgan | News Break', 0.8178364], [2, '0MC46kjl: Examining the reasons for Jonah Williams’ injury and the fallout | News Break', 0.8297063], [3, "0MCYaa4R: Daisy Ridley says she's not in the next Star Wars | News Break", 0.8358507], [4, '0MCdVDMU: FRB: Mystery Radio Signal From Galaxy Four Billion Light-years Away Discovered by Scientists | News Break', 0.8378294], [5, '0MBoXb8e: Tesla loses its VP of production: Electrek | News Break', 0.8402101]]]
[32, [[1, "0MBBLHxJ: Dodgers' magical run interrupted by D-backs | News Break", 0.6481386], [2, '0M85Kg0c: Tiger Woods, children, girlfriend spotted traveling together in Thailand | News Break', 0.7318986], [3, '0MCUWTCO: Twins Have Championship Aspirations, And That Can Only Help Vikings In 2019 | News Break', 0.75617355], [4, '0M5QrsPS: Fighter jet conducts rare flyover of White House in honor of Polish president | News Break', 0.7835147], [5, "0MBvxyKw: Trump blasts Rapinoe: 

[46, [[1, '0MBzdx1l: Nothing But Respect For My President, Megan Rapinoe | News Break', 0.6753983], [2, '0MC7kuHv: Watch: Big Cass Discusses His Battle With Depression, Anxiety | News Break', 0.80179244], [3, '0MCUtt3f: VIDEO: Comparing Premier League teams to NFL franchises | News Break', 0.8150353], [4, '0MBHGmRl: Sophie Turner, Joe Jonas, Priyanka Chopra, and Nick Jonas Have Polar-Opposite Date Night Style | News Break', 0.8252562], [5, '0MBeoSbc: Mueller to testify publicly on July 17 | News Break', 0.8257452]]]
[47, [[1, '0MCPJX3G: Michigan’s World Series dream ends one victory short | News Break', 0.751372], [2, '0MBgsBqL: SpaceX finally caught one of its rocket’s nose cones | News Break', 0.7906407], [3, '0MCcuaV3: Ex-USC soccer coach pleads guilty in admissions scheme | News Break', 0.80507475], [4, "0MCjEJyt: 'The Plagiarists' Ponders Authenticity, Inauthentically | News Break", 0.8144921], [5, '0MCdFP8c: Food52 Puts Its Readers In The Design Chair To Make The Kitchen Tools Of